# Overview
A [swerve drive robot](https://team1640.com/wiki/index.php/Swerve_Central), despite being more mechanically complex, is arguably much simpler for a human to operate than other wheeled mobile robots. This is due to swerve drives being holonomic: they are capable of moving and/or rotating in any direction unlike, for example, a car which can move forwards and backwards but not right and left. Joysticks are naturally well-suited for holonomic control, so they are often used to operate swerve drive robots. It then becomes necessary to introduce a mapping from the space of joystick inputs to the space of the robot's feasible linear and angular velocities. In my experience, it is common to naively implement this mapping as a one-to-one correspondence.

In this notebook, I hope to show some of the drawbacks of the naive bijection mapping and present a novel mapping that avoids these drawbacks. The deviation of the proposed mapping from the naive mapping is more pronounced when the pivots are not equally-distanced and -spaced around the axis of rotation. For this reason, this method may be particularly useful when a robot's drive base is abnormally shaped (e.g. rectangular) or for performing sophisticated manuevers (e.g. operating with an axis-of-rotation fixed at one of the pivots).

# Background
## Controlling a Swerve Drive
A swerve drive consists of some number of pivotable wheels (typically 4) that can be both independently driven and steered, allowing a robot to move holonomically. Controlling a swerve drive robot consists of specifying two vectors at each moment in time:
- the linear velocity $\vec{v} = (v_x, v_y, 0)$ in the ground plane.
- the angular velocity $\vec{\omega} = (0, 0, \omega)$ around some axis of rotation, usually fixed at the centroid of the wheel base.

Given these two control vectors, the velocity for pivot $i$ that is necessary to achieve the desired motion can be uniquely determined using inverse kinematics (see the Appendix for the derivation). Let $\vec{v}_i$ denote the velocity of pivot $i$, let $\theta$ denote the angle the robot makes with the world coordinate frame, let $\vec{r}_i$ denote the position of pivot $i$ with respect to the axis of rotation, and let $R_z(\theta)$ denote the rotation matrix about the z-axis by angle $\theta$. Then the equation that relates pivot $i$'s velocity to the desired robot linear and angular velocity is as follows:
$$\vec{v}_i = \vec{v} + \vec{\omega} \times R_z(\theta) \vec{r}_i, \forall i$$

Since each pivot has a maximum possible speed $\vec{v}_\mathrm{max}$, the pivots constrain the robot's feasible linear and angular velocity as follows:
$$\vec{v} + \vec{\omega} \times R_z(\theta) \vec{r}_i \leq \vec{v}_\mathrm{max}, \forall i$$

Notice the following properties of this constraint:
- $\vec{\omega} = \vec{0} \implies \lVert \vec{v} \rVert \leq v_\mathrm{max}$
- $\vec{v} = \vec{0} \implies \lVert \vec{\omega} \times R_z(\theta) \vec{r}_i \rVert \leq v_\mathrm{max}, \forall i \implies \omega \lVert \vec{r}_i \rVert \leq v_\mathrm{max}, \forall i \implies \omega \leq \frac{v_\mathrm{max}}{\underset{i}{\max} \lVert \vec{r}_i \rVert} =: \omega_\mathrm{max}$

## The Human Interface
Given that swerve drives require two control vectors, it is natural for a swerve drive robot to be operated by a human using two joysticks: one for controlling linear velocity and the other for angular velocity. Assuming the joystick values range from $-1$ to $1$, we can define joystick input vectors as follows:
- the linear joystick vector: $\vec{\mathrm{v}} = (\mathrm{v}_x, \mathrm{v}_y, 0)$, such that $\mathrm{v}_x^2 + \mathrm{v}_y^2 \leq 1$
- the angular joystick vector: $\vec{\mathrm{w}} = (0, 0, \mathrm{w})$, such that $-1 \leq \mathrm{w} \leq 1$

Note that joystick variables which range from -1 to 1 will be denoted in a serif font (e.g. $\vec{\mathrm{v}}$ and $\vec{\mathrm{w}}$) and real variables which are constrained by the robot's physical limitations will be denoted in a script font (e.g. $\vec{v}$ and $\vec{\omega}$).

If we plot the three degrees of freedom in the joystick inputs ($\mathrm{v}_x$, $\mathrm{v}_y$, and $\mathrm{w}$), we discover that the joystick input space is cylindrical.

## Visualizing the Swerve Control Space

The space of feasible linear and angular velocities for the swerve drive, on the other hand, is much more complex as it depends on the number of pivots and their locations with respect to the axis of rotation.